In [ ]:
import sys
sys.path.append("path/to/SWSNet/repo")

In [ ]:
from deconvolution import deconvolve
import matplotlib.pyplot as plt
import numpy as np
from math import pi, sin, cos

In [ ]:
def demean(arr):
    return arr - np.mean(arr)

This is an example code for loading the data. Your data can be saved in any format. Just remember that the radial and transverse components must be in the form of numpy arrays and length of data should not be higher than 40 seconds.

In [ ]:
import h5py

hf = h5py.File('data.h5', 'r')
radial = np.array(hf['radial'])
transverse = np.array(hf['transverse'])
duration = np.array(hf['duration'])
BAZ = np.array(hf['BAZ'])

In [ ]:
import keras
model = keras.models.load_model("../models/SWSNet.h5", compile=False)

In [ ]:
from scipy.signal import resample
from tqdm import tqdm

deconvolved = []

pbar = tqdm(total = len(radial))
for ex in range(len(radial)):
    pbar.update()

    ## resampling the radial and transverse components to make sure it is sampled at 50 Hz; duration is in seconds
    rad = resample(radial[ex], int(duration[ex]) * 50)
    tran = resample(transverse[ex], int(duration[ex]) * 50)
    
    ## performing deconvolution after removing mean from the data
    decon = deconvolve(demean(rad), demean(tran))
    deconvolved.append(decon[1])

In [ ]:
pred = model.predict(np.array(deconvolved))

In [ ]:
alpha_pred = pred[0] * 180
deltat_pred = pred[1] * 2

In [ ]:
def alpha(BAZ, phi):
    if BAZ - phi >= 0 and BAZ - phi <= 180:
        return BAZ - phi
    elif BAZ - phi + 180 >= 0 and BAZ - phi + 180 <= 180:
        return BAZ - phi + 180
    elif BAZ - phi - 180 >= 0 and BAZ - phi - 180 <= 180:
        return BAZ - phi - 180
    else:
        return BAZ - phi - 360

In [ ]:
phi_pred = [alpha(B, p) for B, p in zip(BAZ, alpha_pred -180)]